# Using GN2's REST API

GN2 has a REST API through which the user can fetch data and perform several analyses.
This notebook will include examples for all the currently existing queries, as well as their parameters.

First, we'll start with necessary imports (urllib2 to do HTTP requests with Python and IPython.display to format JSON output).

In [1]:
import urllib2import IPython.display

## Fetching Group/Dataset lists and information

The following query fetches the list of species available in the GN database:

In [1]:
req = urllib2.Request("http://gn2-zach.genenetwork.org/api/v_pre1/species"response = urllib2.urlopen(req)IPython.display.JSON(response.read())